In [1]:
# Install libraries
%pip install numpy
%pip install scipy
%pip install cma
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import cma
from typing import Annotated
import numpy.typing as npt

In [5]:
Vector = Annotated[npt.NDArray[np.float64], (9,)]

# Define system of differential equations
def diff_eq_system(t: float, ini_conditions: Vector) -> Vector:
    a, adot, phi, phidot, theta, thetadot, V_0, m3, nu = ini_conditions

    if abs(a) > 1e200:
        print(f"Warning: a may overflow, a = {a}")

    if abs(adot) > 1e200:
        print(f"Warning: adot may overflow, adot = {a}")

    epsilon = 1e-18

    # Safe powers and divisions using np.power
    nu2 = np.power(nu, 2)

    # Sum epsilon to avoid division by 0
    safe_nu2 = nu2 + epsilon
    safe_a = a + epsilon

    phi2_over_nu2 = np.power(phi, 2) / safe_nu2
    one_minus_phi2_over_nu2 = 1 - phi2_over_nu2
    m3_cubed = np.power(m3, 3)

    # Compute second derivatives with np
    addot = (
        a * (V_0 * np.power(one_minus_phi2_over_nu2, 2) + m3_cubed * theta)
        - 2 * np.power(adot, 2) / safe_a
    )

    phiddot = (
        2 * phi * V_0 / safe_nu2 * one_minus_phi2_over_nu2
        - 3 * adot * phidot / safe_a
    )

    thetaddot = -2 * m3 - 3 * adot * thetadot / safe_a

    result = np.array([adot, addot, phidot, phiddot, thetadot, thetaddot, V_0, m3, nu], dtype=np.float64)

    if np.any(np.isnan(result)):
        raise ValueError("NaN encountered in result, likely due to invalid input or division by zero.")

    return result

# Define cost function to measure how good a solution is
def cost(sol) -> float:
    a_ratio = np.log(sol.y[0][-1] / sol.y[0][0])
    return abs(a_ratio - 60)

checkpoints = np.linspace(0, 1100, 1000)[::20]

def verbose_diff_eq_system(t: float, ini_conditions: Vector) -> Vector:
    result = diff_eq_system(t, ini_conditions)
    if t in checkpoints:
        print(f"t = {t:.5f}")
    return result

# Define objective function
def objective(ini_conditions: Vector) -> float:
    # ini_conditions = np.power(10.0, ini_conditions)
    print(ini_conditions.dtype)
    try:
        sol = solve_ivp(
            fun=diff_eq_system,
            t_span=(0,2000),
            y0=ini_conditions,
            t_eval=np.linspace(0, 2000, 1000),
            method="Radau",
            # max_step=1e-2,
            # rtol=1e-8,
            # atol=1e-10
        )

        if not sol.success:
            return np.nan

        return cost(sol)

    except Exception as e:
        return np.nan
        print("Error:", e)

In [6]:
print(objective(np.array([1e-8, 0, 1, 1e-5, 0.1, 0, 1e-4, 1e-3, 1])))

float64
nan


In [16]:
# Define cma callback function to stop when value is within tolerance
class StopOnTarget:
    def __init__(self, target):
        self.target = target

    def __call__(self, es):
        if es.best.f < self.target:
            print(f"Early stop: f(x) = {es.best.f:.4e} < {self.target}")
            es.stop({"target_f": True})

In [56]:
# Find optimal initial conditions

# Initial guess and step size
x0: Vector = np.array([
    -9, # a0
    0, # adot0
    1, # phi0
    1, # phidot0
    1, # theta0
    1, # thetadot0
    -9, # V_0
    -9, # m3
    1, # nu
], dtype=np.float64)
sigma0 = 1.0
target_callback = StopOnTarget(3)
lower_bounds = [-12] * 9
upper_bounds = [3] * 9

result = cma.fmin(objective, x0, sigma0, callback=target_callback, options={
    "verb_disp": 1,
    "bounds": [lower_bounds, upper_bounds]
})

print("Best solution:", result[0])
print("Objective value:", result[1])

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 9 (seed=1043505, Thu Jun 26 09:29:23 2025)


/tmp/ipykernel_80887/3751651456.py:23: RuntimeWarning: overflow encountered in scalar multiply
  - 2 * np.power(adot, 2) / safe_a
/tmp/ipykernel_80887/3751651456.py:23: RuntimeWarning: overflow encountered in power
  - 2 * np.power(adot, 2) / safe_a
/home/dell_laptop/dev/fisica/.venv/lib/python3.12/site-packages/scipy/integrate/_ivp/rk.py:109: RuntimeWarning: invalid value encountered in divide
  return norm(self._estimate_error(K, h) / scale)
/home/dell_laptop/dev/fisica/.venv/lib/python3.12/site-packages/cma/utilities/utils.py:349: UserWarning:   999 solutions rejected (f-value NaN or None) at iteration 0 (time=Jun 26 09:31:35 2025)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +
/home/dell_laptop/dev/fisica/.venv/lib/python3.12/site-packages/cma/utilities/utils.py:349: UserWarning:   1999 solutions rejected (f-value NaN or None) at iteration 0 (time=Jun 26 09:34:46 2025)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +


KeyboardInterrupt: 

In [49]:
import re
verbosity_options = cma.CMAOptions('verb')

for key, desc in verbosity_options.items():
    match = re.search(r'#\s*(.*)', desc)
    comment = match.group(1) if match else ''
    print(f"{key} → suggested usage: {comment}")


verb_append → suggested usage: initial evaluation counter, if append, do not overwrite output files
verb_disp → suggested usage: v verbosity: display console output every verb_disp iteration
verb_disp_overwrite → suggested usage: v start overwriting after given iteration
verb_filenameprefix → suggested usage: output path (folder) and filenames prefix
verb_log → suggested usage: v verbosity: write data to files every verb_log iteration, writing can be time critical on fast to evaluate functions
verb_log_expensive → suggested usage: allow to execute eigendecomposition for logging every verb_log_expensive iteration, 0 or False for never
verb_plot → suggested usage: v in fmin2(): plot() is called every verb_plot iteration
verb_time → suggested usage: v output timings on console
verbose → suggested usage: v verbosity e.g. of initial/final message, -1 is very quiet, -9 maximally quiet, may not be fully implemented
